<a href="https://colab.research.google.com/github/Amitejashrout99/SuperconductivityPrediction/blob/main/InitialExploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
! ls drive/MyDrive/ProjectDataset/
folder_path = "drive/MyDrive/ProjectDataset/"
training_file_path = "drive/MyDrive/ProjectDataset/train.csv"
#printing the training file path
print( f"The training file path is { training_file_path }" )

train.csv  unique_m.csv
The training file path is drive/MyDrive/ProjectDataset/train.csv


Importing the necessary libraries

In [90]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
#Importing the library which calculates the VIF for the relevant columns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

In [47]:
#Loading the training dataset
atomic_properties_df = pd.read_csv( filepath_or_buffer= training_file_path  )
#Printing the dataframe
print( atomic_properties_df.head() )
#Looks like there are 82 properties

   number_of_elements  mean_atomic_mass  wtd_mean_atomic_mass  \
0                   4         88.944468             57.862692   
1                   5         92.729214             58.518416   
2                   4         88.944468             57.885242   
3                   4         88.944468             57.873967   
4                   4         88.944468             57.840143   

   gmean_atomic_mass  wtd_gmean_atomic_mass  entropy_atomic_mass  \
0          66.361592              36.116612             1.181795   
1          73.132787              36.396602             1.449309   
2          66.361592              36.122509             1.181795   
3          66.361592              36.119560             1.181795   
4          66.361592              36.110716             1.181795   

   wtd_entropy_atomic_mass  range_atomic_mass  wtd_range_atomic_mass  \
0                 1.062396          122.90607              31.794921   
1                 1.057755          122.90607           

In [48]:
#Printing the dataframe properties
print( atomic_properties_df.info() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21263 entries, 0 to 21262
Data columns (total 82 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   number_of_elements               21263 non-null  int64  
 1   mean_atomic_mass                 21263 non-null  float64
 2   wtd_mean_atomic_mass             21263 non-null  float64
 3   gmean_atomic_mass                21263 non-null  float64
 4   wtd_gmean_atomic_mass            21263 non-null  float64
 5   entropy_atomic_mass              21263 non-null  float64
 6   wtd_entropy_atomic_mass          21263 non-null  float64
 7   range_atomic_mass                21263 non-null  float64
 8   wtd_range_atomic_mass            21263 non-null  float64
 9   std_atomic_mass                  21263 non-null  float64
 10  wtd_std_atomic_mass              21263 non-null  float64
 11  mean_fie                         21263 non-null  float64
 12  wtd_mean_fie      

In [49]:
#Getting the co-relation matrix
atomic_properties_df_corr = atomic_properties_df.corr()
#Printing the co-relation matrix
#print( atomic_properties_df_corr.head() )
#Since we are interested in the propertie for the critical_temp only, we will be extracting that feature only
critical_temp_df = atomic_properties_df_corr.loc["critical_temp",:]
#printing the critical_temp_df columns before dropping the critical_temp column
print( critical_temp_df.head() )
#Dropping the critical_temp column and seeing the co-relation values for other
critical_temp_df = critical_temp_df.drop( index=["critical_temp"] )
#printing the columns after dropping the critical_temp column
print( critical_temp_df.index)

number_of_elements       0.601069
mean_atomic_mass        -0.113523
wtd_mean_atomic_mass    -0.312272
gmean_atomic_mass       -0.230345
wtd_gmean_atomic_mass   -0.369858
Name: critical_temp, dtype: float64
Index(['number_of_elements', 'mean_atomic_mass', 'wtd_mean_atomic_mass',
       'gmean_atomic_mass', 'wtd_gmean_atomic_mass', 'entropy_atomic_mass',
       'wtd_entropy_atomic_mass', 'range_atomic_mass', 'wtd_range_atomic_mass',
       'std_atomic_mass', 'wtd_std_atomic_mass', 'mean_fie', 'wtd_mean_fie',
       'gmean_fie', 'wtd_gmean_fie', 'entropy_fie', 'wtd_entropy_fie',
       'range_fie', 'wtd_range_fie', 'std_fie', 'wtd_std_fie',
       'mean_atomic_radius', 'wtd_mean_atomic_radius', 'gmean_atomic_radius',
       'wtd_gmean_atomic_radius', 'entropy_atomic_radius',
       'wtd_entropy_atomic_radius', 'range_atomic_radius',
       'wtd_range_atomic_radius', 'std_atomic_radius', 'wtd_std_atomic_radius',
       'mean_Density', 'wtd_mean_Density', 'gmean_Density',
       'wtd_gmean_

In [50]:
#Getting all the index names where the co-relation value is greater than 0
properties_with_corelation_greater_than_zero = []
for index in critical_temp_df.index:
  if( critical_temp_df[index] > 0 ):
    properties_with_corelation_greater_than_zero.append( index )

#Printing all the indexes where the index value is greater than zero
print( "The index values where the co-relation is greater than zero" )
print( properties_with_corelation_greater_than_zero )
print( "The length of the total number of properties where the co-relation is greater than zero " )
print( len(properties_with_corelation_greater_than_zero ))
#So, out of 82 properties, we have identified 44 properties which are having positive co-relation.
properties_with_corelation_greater_than_zero.append( 'critical_temp' )

The index values where the co-relation is greater than zero
['number_of_elements', 'entropy_atomic_mass', 'wtd_entropy_atomic_mass', 'range_atomic_mass', 'std_atomic_mass', 'wtd_std_atomic_mass', 'mean_fie', 'wtd_mean_fie', 'wtd_gmean_fie', 'entropy_fie', 'wtd_entropy_fie', 'range_fie', 'wtd_range_fie', 'std_fie', 'wtd_std_fie', 'mean_atomic_radius', 'entropy_atomic_radius', 'wtd_entropy_atomic_radius', 'range_atomic_radius', 'std_atomic_radius', 'wtd_std_atomic_radius', 'entropy_Density', 'wtd_entropy_Density', 'range_Density', 'std_Density', 'wtd_std_Density', 'wtd_mean_ElectronAffinity', 'entropy_ElectronAffinity', 'wtd_entropy_ElectronAffinity', 'range_ElectronAffinity', 'wtd_range_ElectronAffinity', 'std_ElectronAffinity', 'wtd_std_ElectronAffinity', 'entropy_FusionHeat', 'wtd_entropy_FusionHeat', 'mean_ThermalConductivity', 'wtd_mean_ThermalConductivity', 'entropy_ThermalConductivity', 'range_ThermalConductivity', 'wtd_range_ThermalConductivity', 'std_ThermalConductivity', 'wtd_s

In [51]:
#It's important to identify which columns are having multi-collinearity or not, if having multi-collinearity, we can try to see if it can be removed
#Filtering the dataframe to have those number of columns only
atomic_properties_filtered = atomic_properties_df[ properties_with_corelation_greater_than_zero ]
#Printing the columns of critical temp filtered head
print( atomic_properties_filtered.info() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21263 entries, 0 to 21262
Data columns (total 45 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   number_of_elements             21263 non-null  int64  
 1   entropy_atomic_mass            21263 non-null  float64
 2   wtd_entropy_atomic_mass        21263 non-null  float64
 3   range_atomic_mass              21263 non-null  float64
 4   std_atomic_mass                21263 non-null  float64
 5   wtd_std_atomic_mass            21263 non-null  float64
 6   mean_fie                       21263 non-null  float64
 7   wtd_mean_fie                   21263 non-null  float64
 8   wtd_gmean_fie                  21263 non-null  float64
 9   entropy_fie                    21263 non-null  float64
 10  wtd_entropy_fie                21263 non-null  float64
 11  range_fie                      21263 non-null  float64
 12  wtd_range_fie                  21263 non-null 

In [52]:
#Getting the VIF columns for the values
atomic_properties_filtered_VIF = pd.DataFrame()
atomic_properties_filtered_VIF["atomic_property"] = atomic_properties_filtered.columns
atomic_properties_filtered_VIF["VIF"] = [variance_inflation_factor( atomic_properties_df_corr.values, i) for i in range( atomic_properties_filtered.shape[1] )]
#Printing the VIF dataframe created
atomic_properties_filtered_VIF["VIF"] = atomic_properties_filtered_VIF.apply( lambda row : "{:f}".format( row["VIF"] ), axis= 1)
print( atomic_properties_filtered_VIF.head(5) )


           atomic_property              VIF
0       number_of_elements   1119462.543833
1      entropy_atomic_mass   9503478.324570
2  wtd_entropy_atomic_mass  37457851.356224
3        range_atomic_mass  14122719.032778
4          std_atomic_mass  54366228.677268


Since, the values were really high and it struck as odd, researching further pointed out a bug/design-flaw in the variance_inflation_librray of the statsmodel.

https://github.com/statsmodels/statsmodels/issues/2376
https://stackoverflow.com/questions/42658379/variance-inflation-factor-in-python

In [57]:
#Adding a constant column so that we can get the actual VIF factor as the VIF factor assumes a constant in the calculation
#atomic_properties_filtered.loc[:,"constant_column"] = 1
#Printing the column values
atomic_properties_filtered_with_constant = add_constant( atomic_properties_filtered )
#Printing the columns in the constant_columns
print( atomic_properties_filtered_with_constant.info() )
atomic_propeprties_multicollinear = pd.Series([variance_inflation_factor(  atomic_properties_filtered_with_constant.values, i)
for i in range(  atomic_properties_filtered_with_constant.shape[1])],
              index=atomic_properties_filtered_with_constant.columns)
#Printing the series
print( atomic_propeprties_multicollinear )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21263 entries, 0 to 21262
Data columns (total 46 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   const                          21263 non-null  float64
 1   number_of_elements             21263 non-null  int64  
 2   entropy_atomic_mass            21263 non-null  float64
 3   wtd_entropy_atomic_mass        21263 non-null  float64
 4   range_atomic_mass              21263 non-null  float64
 5   std_atomic_mass                21263 non-null  float64
 6   wtd_std_atomic_mass            21263 non-null  float64
 7   mean_fie                       21263 non-null  float64
 8   wtd_mean_fie                   21263 non-null  float64
 9   wtd_gmean_fie                  21263 non-null  float64
 10  entropy_fie                    21263 non-null  float64
 11  wtd_entropy_fie                21263 non-null  float64
 12  range_fie                      21263 non-null 

In [61]:
#Getting the value of the collinearity after dropping the values above 100 and seeing how it affects the collinearity value
atomic_propeprties_multicollinear_required_index = [ index for index in atomic_propeprties_multicollinear.index if atomic_propeprties_multicollinear[ index ] <= 100 ]
#Printing the length of the required index values
print( len(atomic_propeprties_multicollinear_required_index) )
#We see that the total_length_value dropped t0 31
#Let's see how teh collinearity is affected now
atomic_properties_df_filtered_second_phase = atomic_properties_filtered_with_constant[ atomic_propeprties_multicollinear_required_index ]
#Adding the constant
atomic_properties_df_filtered_second_phase = add_constant( atomic_properties_df_filtered_second_phase )
atomic_propeprties_multicollinear = pd.Series([variance_inflation_factor(  atomic_properties_df_filtered_second_phase.values, i)
for i in range(  atomic_properties_df_filtered_second_phase.shape[1])],
              index=atomic_properties_df_filtered_second_phase.columns)
#Printing the series
print( atomic_propeprties_multicollinear )

31
const                            800.441472
number_of_elements                26.289322
wtd_entropy_atomic_mass           25.292907
range_atomic_mass                 44.855070
std_atomic_mass                   38.590266
wtd_std_atomic_mass               22.900189
mean_fie                           4.677650
wtd_entropy_fie                   36.022640
wtd_range_fie                     15.968395
mean_atomic_radius                 3.110459
wtd_std_atomic_radius              6.932198
entropy_Density                   26.716855
wtd_entropy_Density               18.969056
range_Density                     42.406825
std_Density                       61.594467
wtd_std_Density                   20.923618
wtd_mean_ElectronAffinity          7.522871
entropy_ElectronAffinity          22.759503
wtd_entropy_ElectronAffinity      19.852400
range_ElectronAffinity            50.363263
wtd_range_ElectronAffinity        15.179439
std_ElectronAffinity              67.624735
wtd_std_ElectronAffinity     

In [66]:
#Taking only those index from the seconnd cycle whose VIF factor is less than 50
properties_essential_for_third_phase = [ index for index in atomic_propeprties_multicollinear.index if atomic_propeprties_multicollinear[ index ] <=50 ]
#Printing the length of the essential properties
print( f"The length of the essential properties is { len( properties_essential_for_third_phase ) }" )
#We can see that the properties have reduced to 28
#Taking those properties only from the main dataframe
atomic_properties_df_filtered_third_phase = atomic_properties_df[ properties_essential_for_third_phase ]
#Adding the constant and then calculating the co-relation values
#Adding the constant
atomic_properties_df_filtered_third_phase = add_constant( atomic_properties_df_filtered_third_phase )
atomic_propeprties_multicollinear = pd.Series([variance_inflation_factor(  atomic_properties_df_filtered_third_phase.values, i)
for i in range(  atomic_properties_df_filtered_third_phase.shape[1])],
              index=atomic_properties_df_filtered_third_phase.columns)
#Printing the series
print( atomic_propeprties_multicollinear )
#Won't reduce any other properties after this as reducing any more features involve including losing the properties of Valence section as well. We will try to see the RMSE error that comes from this value and then
#continue further.

The length of the essential properties is 28
const                            772.092252
number_of_elements                22.801621
wtd_entropy_atomic_mass           23.447467
range_atomic_mass                 43.983955
std_atomic_mass                   33.243984
wtd_std_atomic_mass               20.344569
mean_fie                           4.441361
wtd_entropy_fie                   35.679912
wtd_range_fie                     15.559966
mean_atomic_radius                 3.078526
wtd_std_atomic_radius              6.662701
entropy_Density                   21.585231
wtd_entropy_Density               17.025338
range_Density                     12.623673
wtd_std_Density                   14.067935
wtd_mean_ElectronAffinity          7.432277
entropy_ElectronAffinity          15.223059
wtd_entropy_ElectronAffinity      13.480026
wtd_range_ElectronAffinity        13.965008
wtd_std_ElectronAffinity           4.142970
entropy_FusionHeat                16.533714
wtd_entropy_FusionHeat         

In [84]:
#Extracting these features from the main dataframe and saving it in a csv file
print( atomic_propeprties_multicollinear.index )
#Dropping the const column
atomic_propeprties_multicollinear = atomic_propeprties_multicollinear.loc["number_of_elements":]
#Printing the properties list after dropping the 'const'
print( atomic_propeprties_multicollinear.index )
#Extracting all the required columns from the index for the database
atomic_properties_filtered = atomic_properties_df[ atomic_propeprties_multicollinear.index.tolist() ]
#Printing the columns available in the filtered dataframe
print( atomic_properties_filtered.info() )

Index(['number_of_elements', 'wtd_entropy_atomic_mass', 'range_atomic_mass',
       'std_atomic_mass', 'wtd_std_atomic_mass', 'mean_fie', 'wtd_entropy_fie',
       'wtd_range_fie', 'mean_atomic_radius', 'wtd_std_atomic_radius',
       'entropy_Density', 'wtd_entropy_Density', 'range_Density',
       'wtd_std_Density', 'wtd_mean_ElectronAffinity',
       'entropy_ElectronAffinity', 'wtd_entropy_ElectronAffinity',
       'wtd_range_ElectronAffinity', 'wtd_std_ElectronAffinity',
       'entropy_FusionHeat', 'wtd_entropy_FusionHeat',
       'mean_ThermalConductivity', 'wtd_mean_ThermalConductivity',
       'entropy_ThermalConductivity', 'wtd_range_ThermalConductivity',
       'wtd_std_ThermalConductivity', 'wtd_entropy_Valence', 'critical_temp'],
      dtype='object')
Index(['number_of_elements', 'wtd_entropy_atomic_mass', 'range_atomic_mass',
       'std_atomic_mass', 'wtd_std_atomic_mass', 'mean_fie', 'wtd_entropy_fie',
       'wtd_range_fie', 'mean_atomic_radius', 'wtd_std_atomic_radius

In [89]:
#Splitting the dataset into feature variables and response variable
atomic_properties_filtered_X = atomic_properties_filtered.drop( columns=["critical_temp"] )
atomic_properties_filtered_y = atomic_properties_filtered["critical_temp"]
#printing the feature variable dataset and the response variable dataset
print( "The feature variable dataset is " )
print( atomic_properties_filtered_X.head() )
print( "The response variable dataset is " )
print( atomic_properties_filtered_y.head() )
#Creating the train-test split
atomic_properties_filtered_X_train, atomic_properties_filtered_X_test, atomic_properties_filtered_y_train, atomic_properties_filtered_y_test = train_test_split( atomic_properties_filtered_X,
                                                                                                                                                                atomic_properties_filtered_y, test_size=0.33, random_state=42 )
#Printing the shape of the training and testing dataset so as to ensure that everything is correct
print( f"The shape of the feature variable in the training dataset is {atomic_properties_filtered_X_train.shape}" )
print( f"The shape of the response variable in the training dataset is {atomic_properties_filtered_y_train.shape}")
print( f"The shape of the feature variable in the testing dataset is {atomic_properties_filtered_X_test.shape}" )
print( f"The shape of the response variable in the testing dataset is {atomic_properties_filtered_y_test.shape}")

The feature variable dataset is 
   number_of_elements  wtd_entropy_atomic_mass  range_atomic_mass  \
0                   4                 1.062396          122.90607   
1                   5                 1.057755          122.90607   
2                   4                 0.975980          122.90607   
3                   4                 1.022291          122.90607   
4                   4                 1.129224          122.90607   

   std_atomic_mass  wtd_std_atomic_mass  mean_fie  wtd_entropy_fie  \
0        51.968828            53.622535   775.425         0.791488   
1        47.094633            53.979870   766.440         0.807078   
2        51.968828            53.656268   775.425         0.773620   
3        51.968828            53.639405   775.425         0.783207   
4        51.968828            53.588771   775.425         0.805230   

   wtd_range_fie  mean_atomic_radius  wtd_std_atomic_radius  ...  \
0     735.985714              160.25              69.235569  ..

In [96]:
#Fitting the model on the Linear Regression and trying to get the RMSE score on the test dataset
linearRegressionModel = LinearRegression()
#Training the model
linearRegressionModel.fit( atomic_properties_filtered_X_train, atomic_properties_filtered_y_train )
#Getting the predictions
linearModelPredictions = linearRegressionModel.predict( atomic_properties_filtered_X_test )
#Getting the RMSE scores
mean_squared_error_values = metrics.mean_squared_error( linearModelPredictions, atomic_properties_filtered_y_test, squared= False )
#Converting the RMSE values to a dataframe
#mean_squared_error_values_df = pd.DataFrame( mean_squared_error_values, columns=["error_value"] )
#Printing the dataframe
print( mean_squared_error_values )

19.694334030113463


In [54]:
#printing column names with atomic mass only
for column in atomic_properties_df.columns:
  if( 'atomic_mass' in column ):
    print( column )

mean_atomic_mass
wtd_mean_atomic_mass
gmean_atomic_mass
wtd_gmean_atomic_mass
entropy_atomic_mass
wtd_entropy_atomic_mass
range_atomic_mass
wtd_range_atomic_mass
std_atomic_mass
wtd_std_atomic_mass
